### 문서 로드

In [1]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/KCI_FI003153549.pdf"
loader = PyPDFLoader(file_path)

pages = loader.load()

d:\wsh\langchain-workspace\ch05\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


### 문서 분할

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, # 최대 길이 제한
        chunk_overlap=200 # 정확히 200자를 중첩
    )

splitted_documents = splitter.split_documents(pages)

### 임베딩

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=gemini_api_key)

In [8]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(splitted_documents, embedding_model) # 인메모리(in-memory) 상태

In [15]:
# 영구적인 파일(persistent file)**로 디스크에 저장
# 기존 폴더에 새로운 인덱스 파일을 덮어쓰기 때문에 중복된 파일이 생성되지 않음(항상 가장 마지막에 저장된 벡터스토어의 파일만 존재)
vectorstore.save_local("./faiss_index")

In [9]:
vectorstore

In [16]:
vectorstore = None

In [18]:
# 벡터스토어 재로딩
vectorstore = FAISS.load_local(
    "./faiss_index", # 저장된 FAISS 인덱스 폴더의 경로
    embedding_model,
    allow_dangerous_deserialization=True, #  FAISS 인덱스 내 데이터 역직렬화(deserialization) 허용
)

In [19]:
vectorstore

In [10]:
# 예시 질의
query = "본 연구에서 Private LLM 구축을 위해 수집한 문서의 총 페이지 수와 문서 유형별 비율은 어떻게 되나요?"

In [ ]:
# results = vectorstore.similarity_search(query, k=3) # k는 유사도 검색에서 반환할 상위 문서 개수(top‑k)
results = vectorstore.similarity_search(query, k=5)

for idx, doc in enumerate(results, start=1):
    print(f"[결과 {idx}]\n" + doc.page_content[:300])
    print("---")

[결과 1]
의료기기 임상시험 분야의 도메인 특성에 맞게 튜닝하
기 위해 의료기기 임상시험 전문가로부터 총 158개의 문
서(총 11,954 페이지)를 수집하였다. 수집된 문서는 다음
과 같이 분류된다:
 규제 문서 (30%): FDA, EMA, PMDA 가이드라인, 
GCP 문서 등
 교육 자료 (20%): 임상시험 수행자 교육 매뉴얼, 온라
인 강의 자료 등
 프로토콜 및 보고서 (25%): 임상시험 프로토콜, CSR 
(Clinical Study Report) 템플릿 등
 의료기기 특화 문서 (15%): 의료기기 임상시험 계획
서,
---
[결과 2]
174   Journal of The Korea Society of Computer and Information 
문서(10%)로 구성되며, 각 분류는 도메인 전문가의 검토를 
통해 정확성과 신뢰성을 확보하였다. 특히, 주요 규제 기
관(FDA, EMA, PMDA) 가이드라인, GCP 문서, 환자 동의
서 템플릿 등은 모델이 국제 표준에 기반하여 학습할 수 
있도록 하였고, 교육 자료와 프로토콜은 실질적인 임상시
험 수행과 데이터 관리 작업에서 발생할 수 있는 질문들에 
대응할 수 있는 기초를 제공한다.
이 데이터셋은 다양한 문서 
---
[결과 3]
This study explores the improvement of work efficiency and expertise by applying Private LLM 
based on Large Language Model (LLM) to the field of clinical trials in medical devices. The Private 
LLM system provides sophisticated and accurate answers based on clinical data and shows its potential 
fo
---
[결과 4]
170   Journal of The Korea Society of Computer and Inform